In [1]:
import torch
from agents.bigtwo312 import Agent312
from agents.random import RandomAgent
from env.game import Bigtwo

torch.manual_seed(0)
torch.cuda.manual_seed(0)
game = Bigtwo(42)

agents = [
    Agent312("cuda:0"),
    Agent312("cuda:0"),
    Agent312("cuda:0"),
    Agent312("cuda:0"),
]

In [2]:
for j in range(1000):
    winners = [0, 0, 0, 0]
    losses = [0, 0, 0, 0]
    for i in range(100):
        game.reset()
        while game.winner == None:
            agents[game.player_to_act].act(game)
        for index, agent in enumerate(agents):
            agent.get_reward(game, index)
        winners[game.winner] += 1

    for index, agent in enumerate(agents):
        loss = agent.learn()
        losses[index] = loss

    if j % 250 == 0 and j > 0:
        for index, agent in enumerate(agents):
            agent.save(index)
    print(winners)
    print(losses)

[14, 37, 9, 40]
[0.3476468622684479, 0, 0.2874552011489868, 0]
[8, 43, 5, 44]
[0.20955629646778107, 0, 0.1431303173303604, 0]
[4, 47, 10, 39]
[0.12160143256187439, 0, 0.4062098562717438, 0]
[6, 44, 15, 35]
[0.1692897379398346, 0, 0.45438897609710693, 0]
[11, 33, 25, 31]
[0.2003515660762787, 0, 0.5013326406478882, 0]
[23, 26, 24, 27]
[0.39027395844459534, 0, 0.4389689564704895, 0]
[29, 28, 18, 25]
[0.4252393841743469, 0, 0.31635165214538574, 0]
[25, 25, 30, 20]
[0.3171275854110718, 0, 0.6222104430198669, 0]
[19, 32, 29, 20]
[0.26699280738830566, 0, 0.4501281678676605, 0]
[29, 16, 34, 21]
[0.5274126529693604, 0, 0.5056752562522888, 0]
[26, 17, 37, 20]
[0.40202006697654724, 0, 0.4951411187648773, 0]
[35, 15, 31, 19]
[0.473725289106369, 0, 0.607631504535675, 0]
[41, 15, 27, 17]
[0.6662086248397827, 0, 0.46831175684928894, 0]
[32, 15, 40, 13]
[0.4964134693145752, 0, 0.7359668016433716, 0]


KeyboardInterrupt: 

In [33]:
agents[0].save('initial')

Model and optimizer saved as checkpoints/bigtwo376-initial_20240923_150604.pth


In [3]:
from agents.random import RandomAgent


players = [
    agents[0],
    RandomAgent(),
    RandomAgent(),
    RandomAgent(),
]

for _ in range(5):
    winners = [0, 0, 0, 0]
    losses = [0, 0, 0, 0]
    for i in range(500):
        game.reset()
        while game.winner == None:
            players[game.player_to_act].act(game)
        winners[game.winner] += 1
    print(winners)

[331, 57, 57, 55]
[330, 64, 60, 46]
[335, 61, 48, 56]
[336, 44, 64, 56]
[334, 45, 58, 63]


In [4]:
52 + 52 + 52 * 3 + 52

312